In [10]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.regularizers import l2
import numpy as np

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D, ZeroPadding2D

In [4]:
#create a distiller

class Knowledge_distiller(keras.Model):
    def __init__(self) -> None:
        super().__init__()
        self.teacher=teacher
        self.student=student


    def compile(self,optimizer,metrics,student_loss_fn,distillation_loss_fn,alpha=0.3,temperature=3):
        super().compile(optimizer=optimizer,metrics=metrics)
        self.student_loss_fn=student_loss_fn
        self.distillation_loss_fn=distillation_loss_fn
        self.alpha=alpha
        self.temperature=temperature

In [6]:
# Data Preprocessing
batchsize=32

(x_train,y_train),(x_test,y_test) = keras.datasets.mnist.load_data()

#Scale the data
x_train = x_train.astype("float32") / 255.0
x_train = np.reshape(x_train, (-1, 28, 28, 1))


x_test = x_test.astype("float32") / 255.0
x_test = np.reshape(x_test, (-1, 28, 28, 1))

In [12]:
#create model

# Define the model and set the name to "teacher"
teacher = Sequential(name="teacher")

# Add the first Convolutional Layer with 96 filters, kernel size of 11x11, input shape of (227, 227, 3)
teacher.add(Conv2D(96, (11, 11), input_shape=(28, 28, 1), padding='same'))
teacher.add(Activation('relu'))

# Add MaxPooling Layer with pool size of 3x3 and stride of 2
teacher.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))

# Add ZeroPadding Layer to pad the input with zeros
teacher.add(ZeroPadding2D(padding=(2, 2)))

# Add the second Convolutional Layer with 256 filters and kernel size of 5x5
teacher.add(Conv2D(256, (5, 5), padding='same'))
teacher.add(Activation('relu'))

# Add MaxPooling Layer with pool size of 3x3 and stride of 2
teacher.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))

# Add ZeroPadding Layer to pad the input with zeros
teacher.add(ZeroPadding2D(padding=(1, 1)))

# Add the third Convolutional Layer with 384 filters and kernel size of 3x3
teacher.add(Conv2D(384, (3, 3), padding='same'))
teacher.add(Activation('relu'))

# Add ZeroPadding Layer to pad the input with zeros
teacher.add(ZeroPadding2D(padding=(1, 1)))

# Add the fourth Convolutional Layer with 384 filters and kernel size of 3x3
teacher.add(Conv2D(384, (3, 3), padding='same'))
teacher.add(Activation('relu'))

# Add ZeroPadding Layer to pad the input with zeros
teacher.add(ZeroPadding2D(padding=(1, 1)))

# Add the fifth Convolutional Layer with 256 filters and kernel size of 3x3
teacher.add(Conv2D(256, (3, 3), padding='same'))
teacher.add(Activation('relu'))

# Add MaxPooling Layer with pool size of 3x3 and stride of 2
teacher.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))

# Flatten the output to 1D array
teacher.add(Flatten())

# Add the first Dense layer with 4096 units and Dropout of 0.5
teacher.add(Dense(4096))
teacher.add(Activation('relu'))
teacher.add(Dropout(0.5))

# Add the second Dense layer with 4096 units and Dropout of 0.5
teacher.add(Dense(4096))
teacher.add(Activation('relu'))
teacher.add(Dropout(0.5))

# Add the final Dense layer with 1000 units and softmax activation for classification
teacher.add(Dense(1000))
teacher.add(Activation('softmax'))